In [2]:
import pandas as pd
import sqlite3


In [3]:
orders = pd.read_csv("/content/orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("/content/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("/content/restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()


In [6]:
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [8]:
final_dataset = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)
final_dataset.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_dataset.to_csv("/content/final_food_delivery_dataset.csv", index=False)


In [10]:
df = pd.read_csv("final_food_delivery_dataset.csv")


In [11]:
gold_df = df[df["membership"] == "Gold"]


In [12]:
city_revenue = (
    gold_df
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [13]:
avg_order_value = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

avg_order_value


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [14]:
user_total_spend = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)


In [15]:
high_value_users = user_total_spend[user_total_spend > 1000]


In [16]:
count_users = high_value_users.count()
count_users


np.int64(2544)

In [19]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)


In [20]:
rating_revenue = (
    df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


/tmp/ipython-input-3508619084.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [21]:
gold_df = df[df["membership"] == "Gold"]


In [22]:
city_avg_order_value = (
    gold_df
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

city_avg_order_value


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [23]:
cuisine_stats = (
    df
    .groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(by="distinct_restaurants")
)

cuisine_stats


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [24]:
total_orders = len(df)
total_orders


10000

In [25]:
gold_orders = len(df[df["membership"] == "Gold"])
gold_orders


4987

In [26]:
percentage_gold_orders = (gold_orders / total_orders) * 100
round(percentage_gold_orders)


50

In [34]:
restaurant_stats = (
    df
    .groupby("restaurant_name_x")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)


In [35]:
low_order_restaurants = restaurant_stats[
    restaurant_stats["total_orders"] < 20
]


In [36]:
result = low_order_restaurants.sort_values(
    by="avg_order_value",
    ascending=False
)

result


,total_orders,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [39]:
df["restaurant_name"] = df["restaurant_name_x"].str.strip()


In [40]:
restaurant_stats = (
    df
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "nunique"),
        avg_order_value=("total_amount", "mean")
    )
)


In [41]:
filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]


In [42]:
filtered_sorted = filtered.sort_values(
    by="avg_order_value",
    ascending=False
)

filtered_sorted


,total_orders,avg_order_value
restaurant_name,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [43]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered_sorted.loc[filtered_sorted.index.isin(options)]


,total_orders,avg_order_value
restaurant_name,,
Ruchi Foods Chinese,19,686.603158


In [44]:
combo_revenue = (
    df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [45]:
df["order_date"] = pd.to_datetime(df["order_date"])


/tmp/ipython-input-760424543.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [46]:
df["quarter"] = df["order_date"].dt.quarter


In [47]:
quarter_revenue = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_revenue


,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [48]:

gold_orders = df[df["membership"] == "Gold"]
total_gold_orders = len(gold_orders)
total_gold_orders

4987

In [49]:
hyd_orders = df[df["city"] == "Hyderabad"]
total_revenue_hyd = hyd_orders["total_amount"].sum()
round(total_revenue_hyd)

1889367

In [50]:
distinct_users = df["user_id"].nunique()
distinct_users

2883

In [52]:
gold_orders = df[df["membership"] == "Gold"]
avg_order_value_gold = gold_orders["total_amount"].mean()
round(avg_order_value_gold, 2)

np.float64(797.15)

In [53]:
high_rating_orders = df[df["rating"] >= 4.5]
total_high_rating_orders = len(high_rating_orders)
total_high_rating_orders

3374

In [54]:
gold_df = df[df["membership"] == "Gold"]
city_revenue = gold_df.groupby("city")["total_amount"].sum()
top_city = city_revenue.idxmax()
top_city

'Chennai'

In [55]:
gold_top_city_orders = gold_df[gold_df["city"] == top_city]
total_orders = len(gold_top_city_orders)
total_orders

1337

In [56]:
len(df)

10000